In [15]:
# Data visualization
import numpy as np
import pandas as pd 
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import classification_report, confusion_matrix

import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Describe dataset and Train Model

### 1.1. Describe dataset

In [3]:
# Determine important landmarks for lunge
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [4]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_dataset.csv", sep=',', encoding='utf-8', index=False)


df = describe_dataset("./dataset.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_v', 'right_foot_index_x', 'right_foot_index_y', 'right_foot_index_z', 'right_foot_index_v']
Number of rows: 17040 
Number of columns: 53

Labels: 
M    6171
D    5735
I    5134
Name: label, dtype: int64

Missi

### 1.2. Preprocess data

In [7]:
# load dataset
df = pd.read_csv("./dataset.csv")

# Categorizing label
df.loc[df["label"] == "I", "label"] = 0
df.loc[df["label"] == "M", "label"] = 1
df.loc[df["label"] == "D", "label"] = 2

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 17040 
Number of columns: 53

Labels: 
1    6171
2    5735
0    5134
Name: label, dtype: int64



In [8]:
# Standard Scaling of features
with open("./model/input_scaler.pkl", "rb") as f2:
    input_scaler = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(input_scaler.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

### 1.3. Train model

In [7]:
model = Sequential()
model.add(Dense(52, input_dim = 52, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(52, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(14, activation = "relu"))
model.add(Dense(3, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 52)                2756      
                                                                 
 dropout (Dropout)           (None, 52)                0         
                                                                 
 dense_1 (Dense)             (None, 52)                2756      
                                                                 
 dropout_1 (Dropout)         (None, 52)                0         
                                                                 
 dense_2 (Dense)             (None, 14)                742       
                                                                 
 dense_3 (Dense)             (None, 3)                 45        
                                             

2022-11-14 10:38:36.786157: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-14 10:38:36.786920: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test))

Epoch 1/100


2022-11-14 10:38:48.571461: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-14 10:38:48.827690: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1364/1364 [==============================] - ETA: 0s - loss: 0.1082 - accuracy: 0.9696

2022-11-14 10:39:02.176190: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1364/1364 [==============================] - 15s 11ms/step - loss: 0.1082 - accuracy: 0.9696 - val_loss: 0.0339 - val_accuracy: 0.9947
Epoch 2/100
1364/1364 [==============================] - 13s 10ms/step - loss: 0.0599 - accuracy: 0.9872 - val_loss: 0.0595 - val_accuracy: 0.9815
Epoch 3/100
1364/1364 [==============================] - 13s 10ms/step - loss: 0.0527 - accuracy: 0.9885 - val_loss: 0.0530 - val_accuracy: 0.9944
Epoch 4/100
1364/1364 [==============================] - 13s 10ms/step - loss: 0.0450 - accuracy: 0.9905 - val_loss: 0.0356 - val_accuracy: 0.9938
Epoch 5/100
1364/1364 [==============================] - 13s 10ms/step - loss: 0.1253 - accuracy: 0.9747 - val_loss: 0.0501 - val_accuracy: 0.9921
Epoch 6/100
1364/1364 [==============================] - 14s 10ms/step - loss: 0.0682 - accuracy: 0.9884 - val_loss: 0.0445 - val_accuracy: 0.9865
Epoch 7/100
1364/1364 [==============================] - 14s 10ms/step - loss: 0.0534 - accuracy: 0.9900 - val_loss: 0.0400 - val_

## 2. Model Evaluation

### 2.1. Train set evaluation

In [9]:
predict_x = model.predict(x_test) 
y_pred_class = np.argmax(predict_x, axis=1)

y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)

confusion_matrix(y_test_class, y_pred_class)

  1/107 [..............................] - ETA: 30s

2022-11-14 11:04:43.450300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


107/107 [==============================] - 0s 2ms/step


array([[1037,    7,    1],
       [   0, 1208,    3],
       [   0,    6, 1146]])

In [10]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1045
           1       0.99      1.00      0.99      1211
           2       1.00      0.99      1.00      1152

    accuracy                           1.00      3408
   macro avg       1.00      0.99      1.00      3408
weighted avg       1.00      1.00      1.00      3408



### 2.2. Test set evaluation

In [5]:
test_df = pd.read_csv("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "I", "label"] = 0
test_df.loc[test_df["label"] == "M", "label"] = 1
test_df.loc[test_df["label"] == "D", "label"] = 2

In [13]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(input_scaler.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [16]:
predict_x = model.predict(test_x) 
y_pred_class = np.argmax(predict_x, axis=1)
y_test_class = np.argmax(test_y_cat, axis=1)

confusion_matrix(y_test_class, y_pred_class)

26/26 [==============================] - 0s 2ms/step


array([[267,   0,   0],
       [  0, 263,   0],
       [  0,   0, 299]])

In [17]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       267
           1       1.00      1.00      1.00       263
           2       1.00      1.00      1.00       299

    accuracy                           1.00       829
   macro avg       1.00      1.00      1.00       829
weighted avg       1.00      1.00      1.00       829



## 3. Dump Model

In [11]:
# Dump the best model to a pickle file
with open("./model/lunge_model_deep_learning.pkl", "wb") as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://6d931dd9-8715-41d1-81d4-8f3b853c1109/assets
